Ran shuffles and saves in a separate notebook before splitting test_dataset_from_tfrecords into two functions that did different things.

shuffle_dataset_from_tfrecords is effectively an experimental version of what became build_dataset_from_tfrecords

In [ ]:
import time
import sys
import os
import glob
import math
import threading
import concurrent.futures as cf
import random
import re

import numpy as np
import pandas as pd
import tensorflow as tf
from keras import Input, Model, layers, metrics, losses, callbacks, optimizers, models, utils
from keras import backend as K
import gc
import keras_tuner as kt
from pyfaidx import Fasta

K.clear_session()
gc.collect()

datasets_path = "../../Datasets/"
models_path = "../../Models/"

2025-01-31 00:51:13.839378: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-31 00:51:13.856016: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-31 00:51:13.860967: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-31 00:51:13.873058: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-31 00:51:14.619020: W tensorflow/compiler/tf2

In [2]:
def shuffle_dataset_from_tfrecords(
    tfrecord_pattern,
    batch_size=32,
    compression_type='GZIP',
    shuffle_buffer=100000,
    # shuffle_buffer_small=500
):
    files = tf.data.Dataset.list_files(tfrecord_pattern, shuffle=True)
    
    dataset = files.interleave(
        lambda fname: tf.data.TFRecordDataset(fname, compression_type=compression_type),
        cycle_length=4,        # how many files to read in parallel
        block_length=1,         # how many records to read from each file before switching
        num_parallel_calls=tf.data.AUTOTUNE
)
    
    # Shuffle at the record level
    dataset = dataset.shuffle(shuffle_buffer, reshuffle_each_iteration=True)

    dataset = dataset.batch(batch_size)
    dataset = dataset.shuffle(10*batch_size, reshuffle_each_iteration=True)
    dataset = dataset.unbatch()
    # dataset = dataset.shuffle(shuffle_buffer_small, reshuffle_each_iteration=True)
    
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

In [ ]:
options = tf.io.TFRecordOptions(compression_type="GZIP")
tfrecord_pattern = datasets_path + "Shuffled_again/shuffled_shard_*.tfrecord.gz"

# Shuffled serialized dataset
ssds = shuffle_dataset_from_tfrecords(tfrecord_pattern,
                                  batch_size=32, compression_type='GZIP',
                                  shuffle_buffer=100000)

idx_ssds = ssds.enumerate()

# if not os.path.exists("Last_shuffle"):
#     os.makedirs("Last_shuffle")

# num_shards = 4
# writers = [
#     tf.io.TFRecordWriter(f"{datasets_path}Last_shuffle/shuffled_shard_{i}.tfrecord", options=options)
#     for i in range(num_shards)
# ]

# # Write out round-robin to each shard
# for i, serialized_example in idx_ssds:
#     shard_index = i.numpy() % num_shards
#     writers[shard_index].write(serialized_example.numpy())

# # Close all writers
# for w in writers:
#     w.close()

I0000 00:00:1738309890.744991    1215 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:04:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1738309890.805764    1215 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:04:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1738309890.805825    1215 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:04:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1738309890.810829    1215 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:04:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1738309890.810960    1215 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:04:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0